In [1]:
import pandas as pd

In [2]:
dataset_root = './dataset/training_data.csv'

In [3]:
df = pd.read_csv(dataset_root, sep='\t')

df.columns = ['text', 'label']

In [4]:
g = df.groupby('label')
df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
df

text          label
label                                                             
Non-offensive 0      烏克蘭人呢，就拎槍抗俄嘅你呢啲嘔心左膠呢，就坐飛機打國際線嘅  Non-offensive
              1                            苗栗國不意外事件  Non-offensive
              2                           喜歡語意低能這個詞  Non-offensive
              3                           看來潮人都是熬夜跑  Non-offensive
              4                     不如不要打包汙泥直接丟不是更好  Non-offensive
...                                             ...            ...
Offensive     16725             所以那個該死的孩子被砸成弱智了嗎幹得好      Offensive
              16726                    羞羞臉無下限啊他們，髒髒      Offensive
              16727                         寒假其間低能多      Offensive
              16728                    國會議員別跟一些弱智認真      Offensive
              16729                            戰力單身      Offensive

[33460 rows x 2 columns]

In [5]:
labeled_data = df['text'].tolist()
print(len(labeled_data))
print(labeled_data)

33460
['烏克蘭人呢，就拎槍抗俄嘅你呢啲嘔心左膠呢，就坐飛機打國際線嘅', '苗栗國不意外事件', '喜歡語意低能這個詞', '看來潮人都是熬夜跑', '不如不要打包汙泥直接丟不是更好', '雖然不認識妳，但我想給妳勇氣，吶喊', '曾子懿溼婆真的很可以', '星戰浪喵', '竟敢反對我民進黨看我不抄你家抄光你家三代', '看攀巖大師', '怎麼還是感覺很自閉症原來重點是人很好', '社經智障在臺灣很多啊', '聽說現在找麻煩最大聲的是一個救護呢', '當然要譴責畜生幹不了這麼多的傻子', '廖俊程介紹你宜蘭最好吃的', '送祝福努力爭取讓選民認同', '吹捧老公，然後雅典娜可能要買什麼物品', '妳女兒怎了', '擅長情緒恐嚇的高手', '都什麼五十年代了', '中時不是媒體啊是執筆天地', '你要不要先界定臺派一下是三小，罵你的就叫臺派', '武大郎兄，遇到瘋子喔', '劉泓伭不覺好笑', '多分享這種體育新聞好嗎', '唉真的是一堆連敵人在哪都搞不清楚了的人', '邏輯不通這樣的人也能當體育老師', '再洗一波轉化率，是我實況不死鳥', '你的恐懼感他的最愛', '兄弟也真的是很狠心欸都不建立聯繫', '看妳跟家佳演對手戲好感動', '非弱智，只是不懂得合群的重要性', '哈哈小牛', '因為長得不討喜就這樣對待他好可惡啊完完全全的霸凌', '之前爛片生肖的開工率', '那個一言不合就驅離人的一言堂還沒消失啊', '吳他家才是金門人民的悲哀、親共的狗兒子', '粒就撐了還是年輕有戰鬥能力', '你是誰這段話，好在意', '搶手紙的也是', '對啊你的境界真的是處女無法想像的地平線啊', '由此可知你在人家的思緒裡有糖份', '先去喝咖啡啦，，為這種奧咖生氣不值得', '真是偉大的父愛而且能發揚到愛有殘疾的人，令人敬佩，阿彌陀佛', '是要他選外交部長嗎', '臺灣人的你不該管到苗栗國的事恐引發外交債務危機', '支持妳晚上美滿', '惡兆的延遲北派真的打到我懷疑人生想到無名王者', '你是法務部長都講幹話，臺灣法制沒救', '把權杖還人家啦', '有智慧的女孩子', '柯文哲：垃圾不分藍綠', '娘娘的手好乖巧', '香港變成今時今日個樣，都有你一份大功', "林先生'你的大愛太偉大了老天會護持你", '你在期盼中夭什麼', '臺灣地區這批紅統細胞一一露

In [6]:
import torch
import numpy as np
from transformers import BertTokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
labels = {'Offensive':0,
          'Non-offensive':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 32, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [8]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

26768 3346 3346


In [9]:
from torch import nn
from transformers import BertModel

In [10]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [11]:
output_model = './models/model_bert.pth'

# save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

In [12]:
from torch.optim import Adam
from tqdm import tqdm

In [13]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            save(model, optimizer)
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [14]:
EPOCHS = 3

model = BertClassifier()
LR = 3e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████████████| 13384/13384 [1:09:19<00:00,  3.22it/s]


Epochs: 1 | Train Loss:  0.121                 | Train Accuracy:  0.894                 | Val Loss:  0.048                 | Val Accuracy:  0.966


100%|███████████████████████████████████████████████████████████████████████████████████████████| 13384/13384 [1:32:53<00:00,  2.40it/s]


Epochs: 2 | Train Loss:  0.020                 | Train Accuracy:  0.987                 | Val Loss:  0.037                 | Val Accuracy:  0.977


100%|███████████████████████████████████████████████████████████████████████████████████████████| 13384/13384 [1:07:50<00:00,  3.29it/s]


Epochs: 3 | Train Loss:  0.008                 | Train Accuracy:  0.995                 | Val Loss:  0.043                 | Val Accuracy:  0.977


In [15]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [16]:
evaluate(model, df_test)

Test Accuracy:  0.980
